In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math

In [2]:
hankel = Hankel.Hankel()
rank = Rank.Rank()
cluster = Cluster.Cluster()
meepc = Meepc.MEEPC()

In [3]:
df = pd.read_csv('data/training_data.csv')
sensors = [3,9,25,43]
df = df.iloc[:, sensors]

In [4]:
X = df.iloc[:,1].values
X = hankel.fit(X,30,0.5)
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
for i in range(optimal_k):
    cluster = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(X)
    U,Sigma,VT = np.linalg.svd(cluster)
    V = VT.T
    cluster = np.matmul(cluster,V[:,:r])
    weight,center = meepc.fit(cluster)
    var1=np.square(cluster-center)
    var2=np.matmul(weight,var1.T)
    max_radius=np.amax(var2)
    radiis.append(max_radius)
radiis



5


[1.000009624398698,
 1.0000037786508869,
 0.9999983630446033,
 1.0000096243986938,
 1.000006924729706]

In [5]:
X = df.iloc[:,1].values
X = hankel.fit(X,30,0.5)
r = rank.fit(X)
U,Sigma,VT = np.linalg.svd(X)
V = VT.T
X = np.matmul(X,V[:,:r])
weight,center = meepc.fit(X)
var1=np.square(X-center)
var2=np.matmul(weight,var1.T)
max_radius = math.sqrt(np.amax(var2))
max_radius


1.0008168366684211

In [4]:
df_test = pd.read_csv('data/testdata.csv')
df_test = df_test.iloc[:, sensors]
X_test = df_test[:1].values

In [5]:
X_test = hankel.fit(X_test,30,0.5)
r = rank.fit(X_test)
if r==1:
    r = 2
U,Sigma,VT = np.linalg.svd(X_test)
V = VT.T
X_test = np.matmul(X_test,V[:,:r])
weight,center = meepc.fit(X_test)
var1=np.square(X_test-center)
var2=np.matmul(weight,var1.T)
max_radius = math.sqrt(np.amax(var2))
max_radius

1.0043296678240774

In [9]:
if math.isnan(max_radius):
        print('Rank: ',r)
        print('Cluster Shape: ',cluster.shape)
        print('CLuster: ',cluster)
        print('Weight Shape: ', weight.shape)
        print('Weight: ', weight)
        print('Center Shape: ', center.shape)
        print('Center: ', center)
        print('Var1: ', var1)
        print('Var2: ', var2)